In [1]:
import os
import torch
import soundfile as sf
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch.optim as optim
import librosa
from tqdm import tqdm
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
from sklearn.preprocessing import StandardScaler
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from torch.utils.data import DataLoader, TensorDataset
from sklearn.manifold import TSNE
from torchsummary import summary
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix

In [2]:
# Check if PyTorch is installed properly
print("PyTorch version:", torch.__version__)
# Check if CUDA (GPU support) is available
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)
if cuda_available:
    # Check the GPU device name
    gpu_name = torch.cuda.get_device_name(0)
    print("GPU device name:", gpu_name)
    # Check the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print("Number of available GPUs:", num_gpus)
else:
    print("CUDA is not available. PyTorch is using the CPU.")
    
    
print("Available GPUs: ", torch.cuda.device_count())
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

PyTorch version: 2.4.0
CUDA available: True
GPU device name: NVIDIA GeForce RTX 4090
Number of available GPUs: 1
Available GPUs:  1
Using device: cuda


In [5]:
# W2V
# XLSR
# Set paths to feature files
data_dir = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\Features\\SSL"
data_dir1 = "F:\\Awais_data\\Datasets\\ASV21\\Features\\SSL\\W2V\\DF\\Hidden"
X_train_hidden_file = os.path.join(data_dir, "W2V_Train_hidden_states_features.npy")
X_val_hidden_file = os.path.join(data_dir, "W2V_dev_hidden_states_features.npy")
X_test_hidden_file = os.path.join(data_dir1, "W2V_DF_hidden_states_features01.npy")

# X_train_cnn_file = os.path.join(data_dir, "W2V_Train_features_last_cnn_layer.npy")
# X_val_cnn_file = os.path.join(data_dir, "W2V_dev_features_last_cnn_layer.npy")
# X_test_cnn_file = os.path.join(data_dir1, "W2V_DF_features_last_cnn_layer12.npy")

y_train_file = os.path.join(data_dir, "W2V_Train_labels.npy")
y_val_file = os.path.join(data_dir, "W2V_dev_labels.npy")
y_test_file = os.path.join(data_dir1, "W2V_DF_labels01.npy")

In [6]:
# Load features and labels
def load_features(file_path):
    return np.load(file_path)

# Function to split the tensor across multiple GPUs
def split_tensor_across_gpus(tensor, devices):
    split_size = tensor.size(0) // len(devices)
    tensor_splits = [tensor[i * split_size:(i + 1) * split_size].to(devices[i]) for i in range(len(devices))]
    
    # Handle any remainder by adding the last remaining elements to the last split
    if tensor.size(0) % len(devices) != 0:
        tensor_splits[-1] = torch.cat([tensor_splits[-1], tensor[len(devices) * split_size:].to(devices[-1])])
    
    return tensor_splits

# Automatically detect available GPUs
num_gpus = torch.cuda.device_count()
devices = [torch.device(f'cuda:{i}') for i in range(num_gpus)]

print(f"Detected {num_gpus} GPU(s): {devices}")

print("Loading features and labels...")
with tqdm(total=6) as pbar:
    # # Load and split tensors across all available GPUs
    X_train_hidden = torch.tensor(load_features(X_train_hidden_file), dtype=torch.float32)
    X_train_splits = split_tensor_across_gpus(X_train_hidden, devices)
    pbar.update(1)

    X_val_hidden = torch.tensor(load_features(X_val_hidden_file), dtype=torch.float32)
    X_val_splits = split_tensor_across_gpus(X_val_hidden, devices)
    pbar.update(1)

    X_test_hidden = torch.tensor(load_features(X_test_hidden_file), dtype=torch.float32)
    X_test_splits = split_tensor_across_gpus(X_test_hidden, devices)
    pbar.update(1)

    y_train = torch.tensor(load_features(y_train_file), dtype=torch.float32)
    y_train_splits = split_tensor_across_gpus(y_train, devices)
    pbar.update(1)

    y_val = torch.tensor(load_features(y_val_file), dtype=torch.float32)
    y_val_splits = split_tensor_across_gpus(y_val, devices)
    pbar.update(1)

    y_test = torch.tensor(load_features(y_test_file), dtype=torch.float32)
    y_test_splits = split_tensor_across_gpus(y_test, devices)
    pbar.update(1)

Detected 1 GPU(s): [device(type='cuda', index=0)]
Loading features and labels...


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:18<00:00, 13.10s/it]


In [7]:
devices = [torch.device('cuda:0'), torch.device('cuda:1'), torch.device('cuda:2')]

In [8]:
# # Print shapes and sizes
print("Training Hidden Features Shape:", X_train_hidden.shape)
print("Validation Hidden Features Shape:", X_val_hidden.shape)
print("Test Hidden Features Shape:", X_test_hidden.shape)
# print("Training CNN Features Shape:", X_train_cnn.shape)
# print("Validation CNN Features Shape:", X_val_cnn.shape)
# print("Test CNN Features Shape:", X_test_cnn.shape)
print("Training Labels Shape:", y_train.shape)
print("Validation Labels Shape:", y_val.shape)
print("Test Labels Shape:", y_test.shape)

Training Hidden Features Shape: torch.Size([25380, 49, 1024])
Validation Hidden Features Shape: torch.Size([24844, 49, 1024])
Test Hidden Features Shape: torch.Size([40000, 199, 1024])
Training Labels Shape: torch.Size([25380])
Validation Labels Shape: torch.Size([24844])
Test Labels Shape: torch.Size([40000])


In [9]:
# Reshape features with progress bar
print("Reshaping features...")
with tqdm(total=3) as pbar:
    X_train_hidden = X_train_hidden.view(X_train_hidden.shape[0], -1)
    pbar.update(1)
    X_val_hidden = X_val_hidden.view(X_val_hidden.shape[0], -1)
    pbar.update(1)
    X_test_hidden = X_test_hidden.view(X_test_hidden.shape[0], -1)
    pbar.update(1)
    # X_train_cnn = X_train_cnn.view(X_train_cnn.shape[0], -1)
    # pbar.update(1)
    # X_val_cnn = X_val_cnn.view(X_val_cnn.shape[0], -1)
    # pbar.update(1)
    # X_test_cnn = X_test_cnn.view(X_test_cnn.shape[0], -1)
    # pbar.update(1)

Reshaping features...


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 999.75it/s]


In [10]:

X_train = X_train_hidden
X_val = X_val_hidden
X_test = X_test_hidden

# X_train = X_train_cnn
# X_val = X_val_cnn
# X_test = X_test_cnn

In [11]:
print("Before normalization")
print(X_train[0])
print(y_train[0])
print(X_val[0])
print(y_val[0])
print(X_test[0])
print(y_test[0])
print("Shapes")
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

Before normalization
tensor([-0.1370, -0.0350, -0.0693,  ..., -1.1063,  0.6121,  0.1675])
tensor(0.)
tensor([ 0.2082,  0.2715, -0.0374,  ..., -0.2409,  0.2648,  0.1941])
tensor(0.)
tensor([-0.6501,  0.5213,  0.1614,  ..., -0.6026,  0.7914,  0.4863])
tensor(1.)
Shapes
torch.Size([25380, 50176])
torch.Size([25380])
torch.Size([24844, 50176])
torch.Size([24844])
torch.Size([40000, 203776])
torch.Size([40000])


In [12]:
# Clear GPU cache
torch.cuda.empty_cache()

In [13]:
import torch

def compute_mean_std(X, batch_size, device):
    num_features = X.size(1)
    mean = torch.zeros(num_features, device=device)
    std = torch.zeros(num_features, device=device)
    
    num_batches = (X.size(0) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch = X[i*batch_size:(i+1)*batch_size].to(device)
        mean += batch.mean(dim=0)
        std += batch.std(dim=0)
    
    mean /= num_batches
    std /= num_batches
    
    return mean, std

def standardize_in_chunks(X, mean, std, batch_size, device):
    num_batches = (X.size(0) + batch_size - 1) // batch_size
    standardized_chunks = []
    for i in range(num_batches):
        batch = X[i*batch_size:(i+1)*batch_size].to(device)
        standardized_batch = (batch - mean) / std
        standardized_chunks.append(standardized_batch.cpu())
    
    return torch.cat(standardized_chunks, dim=0)

print("Standardizing features...")
batch_size = 500  # Adjust batch size based on available memory

# Load data to CPU
X_train_cpu = X_train.cpu()
X_val_cpu = X_val.cpu()
X_test_cpu = X_test.cpu()

# Compute mean and std
mean, std = compute_mean_std(X_train_cpu, batch_size, device)
mean, std = compute_mean_std(X_val_cpu, batch_size, device)
mean1, std1 = compute_mean_std(X_test_cpu, batch_size, device)

# Standardize in chunks
X_train= standardize_in_chunks(X_train_cpu, mean, std, batch_size, device)
X_val= standardize_in_chunks(X_val_cpu, mean, std, batch_size, device)
X_test= standardize_in_chunks(X_test_cpu, mean1, std1, batch_size, device)

print("Processing complete.")


Standardizing features...
Processing complete.


In [14]:
print("After normalization")
print(X_train[0])
print(y_train[0])
print(X_val[0])
print(y_val[0])
print(X_test[0])
print(y_test[0])
print("Shapes")
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

After normalization
tensor([ 0.6847, -1.1545, -0.1941,  ..., -1.9798,  0.4763,  0.3751])
tensor(0.)
tensor([ 2.1590,  1.0083,  0.0574,  ..., -0.1239, -0.5435,  0.4577])
tensor(0.)
tensor([-0.0172,  1.6833,  0.6449,  ...,  0.0727, -0.2328,  1.0112])
tensor(1.)
Shapes
torch.Size([25380, 50176])
torch.Size([25380])
torch.Size([24844, 50176])
torch.Size([24844])
torch.Size([40000, 203776])
torch.Size([40000])


Model training and validation

In [15]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [16]:
# Hyperparameters as a configuration dictionary
config = {
    'hidden_dim': 512,
    'output_dim': 1,  # Assuming binary classification
    'num_epochs': 100,
    'batch_size': 128,
    'learning_rate': 0.0001,
    'model_save_path': 'W2V_best_mlp_crossmodel_asv21_DF_cnn01'  # Base path for saving the model
}

In [17]:
# Automatically determine input_dim from training data
train_input_dim = X_train.shape[1]
test_input_dim = X_test.shape[1]

In [18]:
def create_dataloader(X, y, batch_size):
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

train_val_loader = create_dataloader(torch.cat((X_train, X_val)), torch.cat((y_train, y_val)), config['batch_size'])
test_loader = create_dataloader(X_test, y_test, config['batch_size'])

In [19]:
# Initialize model for training and validation
train_model = MLPClassifier(train_input_dim, config['hidden_dim'], config['output_dim']).cuda()
# Initialize model for testing
test_model = MLPClassifier(test_input_dim, config['hidden_dim'], config['output_dim']).cuda()

criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(train_model.parameters(), lr=config['learning_rate'])

In [20]:
# Training function
def train_model_fn(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(dataloader, desc="Training", leave=False):
        X_batch, y_batch = X_batch.cuda(), y_batch.float().cuda()  # Ensure target is float for BCEWithLogitsLoss
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Evaluation function
def evaluate_model_fn(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    all_labels = []
    all_outputs = []
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for batch_idx, (X_batch, y_batch) in enumerate(tqdm(dataloader, desc="Evaluation", leave=False)):
            X_batch, y_batch = X_batch.cuda(), y_batch.float().cuda()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            all_labels.extend(y_batch.cpu().numpy())
            all_outputs.extend(torch.sigmoid(outputs).cpu().numpy())  # Use sigmoid to convert logits to probabilities

            # Debug information: print batch index and batch size
            if batch_idx % 100 == 0:  # Print every 100 batches
                print(f"Processed batch {batch_idx}/{num_batches}, batch size: {len(X_batch)}")
    
    # Calculate metrics
    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)
    
    # Accuracy
    predictions = [1 if x > 0.5 else 0 for x in all_outputs]
    accuracy = accuracy_score(all_labels, predictions)
    
    # Precision, Recall, F1
    precision = precision_score(all_labels, predictions, zero_division=1)
    recall = recall_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)
    
    # AUC
    auc = roc_auc_score(all_labels, all_outputs)
    
    # EER Calculation
    fpr, tpr, thresholds = roc_curve(all_labels, all_outputs)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
    eer = fpr[np.nanargmin(np.abs(fnr - fpr))]
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, predictions)
    
    return total_loss / num_batches, accuracy, precision, recall, f1, auc, eer, cm


In [21]:
# Training loop
epochs_no_improve = 0
n_epochs_stop = 10
best_val_loss = float('inf')
best_val_eer = float('inf')
for epoch in range(config['num_epochs']):
    print(f'Starting epoch {epoch+1}/{config["num_epochs"]}')
    train_loss = train_model_fn(train_model, train_val_loader, criterion, optimizer)
    
    # Validate using the validation set
    val_loss, val_accuracy, val_precision, val_recall, val_f1, val_auc, val_eer, cm = evaluate_model_fn(train_model, create_dataloader(X_val, y_val, config['batch_size']), criterion)
    
    print(f'Epoch {epoch+1}/{config["num_epochs"]}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val AUC: {val_auc:.4f}, Val EER: {val_eer:.4f}')
       
    if val_eer < best_val_eer:
        best_val_eer = val_eer
        epochs_no_improve = 0
        
        # Remove the previously saved model if it exists
        if os.path.exists(config['model_save_path']):
            os.remove(config['model_save_path'])
        
        # Save the current model
        torch.save(train_model.state_dict(), config['model_save_path'])
        print(f'Saved best model with Val EER: {val_eer:.4f} to {config["model_save_path"]}')
    else:
        epochs_no_improve += 1
    
    # Early stopping
    if epochs_no_improve >= n_epochs_stop:
        print(f'Early stopping at epoch {epoch+1}')
        break

Starting epoch 1/100


Evaluation:   4%|██▊                                                                   | 8/195 [00:00<00:04, 37.79it/s]

Processed batch 0/195, batch size: 128


Evaluation:  55%|█████████████████████████████████████▎                              | 107/195 [00:02<00:02, 38.75it/s]

Processed batch 100/195, batch size: 128


...

Evaluation:   6%|████▏                                                               | 12/195 [00:00<00:01, 119.09it/s]

Processed batch 0/195, batch size: 128


Evaluation:  63%|██████████████████████████████████████████▎                        | 123/195 [00:00<00:00, 139.47it/s]

Processed batch 100/195, batch size: 128


Epoch 100/100, Train Loss: 0.0377, Val Loss: 0.0265, Val Accuracy: 0.9884, Val Precision: 0.9876, Val Recall: 0.9997, Val F1: 0.9936, Val AUC: 0.9995, Val EER: 0.0126


In [22]:
def preprocess_features(X, target_dim):
    # Reshape features to match the target dimension
    if X.shape[1] != target_dim:
        # Resize or truncate features
        X = X[:, :target_dim]
    return X

# Preprocess features
target_dim = X_train.shape[1]
X_test_processed = preprocess_features(X_test_hidden, target_dim)


In [23]:
test_loader = create_dataloader(X_test_processed, y_test, config['batch_size'])

In [24]:

# Ensure same input dimension is used for testing
if os.path.exists(config['model_save_path']):
    test_model = MLPClassifier(train_input_dim, config['hidden_dim'], config['output_dim']).cuda()
    test_model.load_state_dict(torch.load(config['model_save_path']))
    print(f'Loaded best model from {config["model_save_path"]} for testing')

    # Testing the model
    test_loss, test_accuracy, test_precision, test_recall, test_f1, test_auc, test_eer, test_cm = evaluate_model_fn(test_model, test_loader, criterion)
    # print(f'Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print(f'Test Precision: {test_precision:.4f}')
    print(f'Test Recall: {test_recall:.4f}')
    print(f'Test F1: {test_f1:.4f}')
    print(f'Test AUC: {test_auc:.4f}')
    print(f'Test EER: {test_eer:.4f}')
    print(f'Test Confusion Matrix:\n{test_cm}')
else:
    print(f"No saved model found at {config['model_save_path']}")



C:\Users\mawais\AppData\Local\Temp\ipykernel_26932\1080504748.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_model.load_state_dict(torch.load(config['model_save_pa

Loaded best model from W2V_best_mlp_crossmodel_asv21_DF_cnn01 for testing


Evaluation:   1%|▋                                                                     | 3/313 [00:00<00:14, 20.85it/s]

Processed batch 0/313, batch size: 128


Evaluation:  33%|██████████████████████▌                                             | 104/313 [00:03<00:08, 25.89it/s]

...

Evaluation:  65%|████████████████████████████████████████████▎                       | 204/313 [00:08<00:06, 17.36it/s]

Processed batch 200/313, batch size: 128


Evaluation:  97%|██████████████████████████████████████████████████████████████████  | 304/313 [00:14<00:00, 18.34it/s]

Processed batch 300/313, batch size: 128


Test Accuracy: 0.8883
Test Precision: 0.9634
Test Recall: 0.9189
Test F1: 0.9406
Test AUC: 0.7354
Test EER: 0.2794
Test Confusion Matrix:
[[  169  1345]
 [ 3123 35363]]


In [25]:
# Hyperparameters as a configuration dictionary
config = {
    'hidden_dim': 512,
    'output_dim': 1,  # Assuming binary classification
    'num_epochs': 100,
    'batch_size': 128,
    'learning_rate': 0.0001,
    'model_save_path': 'XLSR_best_mlp_crossmodel_asv21_DF_cnn03'  # Base path for saving the model
}

In [1]:
import os
import pandas as pd

def convert_csv_to_data(csv_file_path):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Generate the output .data file path
    data_file_path = csv_file_path.replace('.csv', '.data')
    
    # Save the DataFrame as a .data file (same format as CSV, just different extension)
    df.to_csv(data_file_path, index=False, header=True, sep=',')  # Use comma as delimiter

    print(f"Converted {csv_file_path} to {data_file_path}")

# Example usage
csv_file = "C:\\Notebooks\\rrl\\train_utterance_MDSTD_processed.csv"  # Path to the CSV file
convert_csv_to_data(csv_file)


Converted C:\Notebooks\rrl\train_utterance_MDSTD_processed.csv to C:\Notebooks\rrl\train_utterance_MDSTD_processed.data


In [4]:
import os
import pandas as pd

def convert_csv_to_data(csv_file_path):
    # Load the CSV file into a DataFrame (no header assumed here)
    df = pd.read_csv(csv_file_path, header=None)  
    
    # Generate the output .data file path
    data_file_path = csv_file_path.replace('.csv', '.data')
    
    # Define the desired precision for each column
    precision_map = {
        45: '%.9f',  # Column 0 with 3 decimal places
        46: '%.9f',  # Column 1 with 3 decimal places
        47: '%.9f',  # Column 2 with 3 decimal places
        # You can adjust column index and precision here
        48: '%.9e',  # Column 13 with scientific notation
        49: '%.9f',  # Column 14 with 3 decimal places
        50: '%.9f',  # Column 0 with 3 decimal places
        51: '%.9f',  # Column 1 with 3 decimal places
        52: '%.9f',  # Column 2 with 3 decimal places
        # You can adjust column index and precision here
        53: '%.9e',  # Column 13 with scientific notation
        54: '%.9f',  # Column 14 with 3 decimal places
        55: '%.9f',  # Column 0 with 3 decimal places
        56: '%.9f',  # Column 1 with 3 decimal places
        57: '%.9f',  # Column 2 with 3 decimal places
        # You can adjust column index and precision here
        58: '%.9e',  # Column 13 with scientific notation
        59: '%.9f',  # Column 14 with 3 decimal places
    }
    
    # Format each column based on the precision map
    formatted_columns = []
    for col in df.columns:
        if col in precision_map:
            formatted_column = df[col].map(lambda x: precision_map[col] % x if pd.notnull(x) else '')
        else:
            # Default to 3 decimal places if no precision specified
            formatted_column = df[col].map(lambda x: '%.3f' % x if pd.notnull(x) else '')
        
        formatted_columns.append(formatted_column)
    
    # Concatenate all formatted columns back into a DataFrame
    formatted_df = pd.concat(formatted_columns, axis=1)
    
    # Save the formatted DataFrame to a .data file
    formatted_df.to_csv(data_file_path, index=False, header=False, sep=',', quoting=None)
    
    print(f"Converted {csv_file_path} to {data_file_path}")

# Example usage
csv_file = "C:\\Notebooks\\rrl_source\\dataset_raw\\train_segment_MDSTD_processed.csv"  # Path to the CSV file
convert_csv_to_data(csv_file)



Converted C:\Notebooks\rrl\train_utterance_MDSTD_processed.csv to C:\Notebooks\rrl\train_utterance_MDSTD_processed.data


In [ ]:
python3 experiment.py -d train_fulldf_mfcc -bs 32 -s 1@1024 -e200 -lrde 200 -lr 0.0002 -ki 0 -i 0 -wd 0.0001 --nlaf --alpha 0.999 --beta 8 --gamma 1 --temp 0.01 --print_rule --save_best --skip --use_not  --estimated_grad --weighted  &

In [2]:
import os
import pandas as pd

def convert_csv_to_data(csv_file_path):
    # Load the CSV file into a DataFrame (no header assumed here)
    df = pd.read_csv(csv_file_path, header=None)  
    
    # Generate the output .data file path
    data_file_path = csv_file_path.replace('.csv', '.data')
    
    # Format columns based on conditions
    formatted_columns = []
    for col in df.columns:
        if col >= 1024 and col <= 1025:
            # Retain full precision for columns 44 to 59 (no formatting change)
            formatted_column = df[col]
        else:
            # Apply 3 decimal places for other columns
            formatted_column = df[col].map(lambda x: '%.3f' % x if pd.notnull(x) else '')
        
        formatted_columns.append(formatted_column)
    
    # Concatenate all formatted columns back into a DataFrame
    formatted_df = pd.concat(formatted_columns, axis=1)
    
    # Save the formatted DataFrame to a .data file
    formatted_df.to_csv(data_file_path, index=False, header=False, sep=',', quoting=None)
    
    print(f"Converted {csv_file_path} to {data_file_path}")

# Example usage
csv_file = "C:\\Notebooks\\rrl_source\\dataset_raw\\train_segment_Wav2Vec2_part1_processed.csv"  # Path to the CSV file
convert_csv_to_data(csv_file)


Converted C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part1_processed.csv to C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part1_processed.data


In [3]:
import os

# Define the directory path where you want to save the file
directory_path = "C:\\Notebooks\\rrl_source\\dataset\\"  # Change this to your desired path

# Ensure the directory exists
os.makedirs(directory_path, exist_ok=True)

# Define the full file path
file_path = os.path.join(directory_path, "train_segment_Wav2Vec2_part1_processed.INFO")

# Open the file in write mode
with open(file_path, "w") as file:
    # Write 1024 continuous features
    for i in range(1, 1025):
        file.write(f"{i} continuous\n")
    
    # Write the remaining lines
    file.write("class discrete\n")
    file.write("LABEL_POS -1\n")

print(f"{file_path} has been created successfully.")


C:\Notebooks\rrl_source\dataset\train_segment_Wav2Vec2_part1_processed.INFO has been created successfully.


In [1]:
import pandas as pd

# Path to the large CSV file
file_path = r"C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_1024.csv"

# Parameters for subpart size
output_size = 80000  # Total rows per output file
samples_per_label = output_size // 2  # 40,000 samples for each label

# Initialize counters
part_num = 1
remaining_ones = pd.DataFrame()
remaining_zeros = pd.DataFrame()

# Read the entire CSV file in one go to determine total rows
total_rows = sum(1 for _ in open(file_path)) - 1  # Subtracting 1 for header row
print(f"Total rows in the original file: {total_rows}")

# Process the file in chunks to avoid loading the entire file into memory at once
for chunk in pd.read_csv(file_path, chunksize=total_rows):
    # Separate data by label
    ones = chunk[chunk['Label'] == 1]
    zeros = chunk[chunk['Label'] == 0]

    # Concatenate with any remaining data from the previous chunk
    ones = pd.concat([remaining_ones, ones], ignore_index=True)
    zeros = pd.concat([remaining_zeros, zeros], ignore_index=True)
    
    # Process while there are enough samples for a balanced subpart
    while len(ones) >= samples_per_label and len(zeros) >= samples_per_label:
        # Sample 40,000 from each label for a balanced subset
        balanced_chunk = pd.concat([ones[:samples_per_label], zeros[:samples_per_label]])
        
        # Remove used samples from ones and zeros
        ones = ones.iloc[samples_per_label:].reset_index(drop=True)
        zeros = zeros.iloc[samples_per_label:].reset_index(drop=True)
        
        # Shuffle the balanced data
        balanced_chunk = balanced_chunk.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Save to a new CSV file
        output_file = rf"C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part{part_num}.csv"
        balanced_chunk.to_csv(output_file, index=False)
        print(f"Saved {output_file} with {len(balanced_chunk)} samples.")
        
        part_num += 1

    # Store any remaining samples for the next loop iteration
    remaining_ones = ones
    remaining_zeros = zeros

# Handle the final part with remaining data if any
if not remaining_ones.empty and not remaining_zeros.empty:
    final_chunk = pd.concat([remaining_ones, remaining_zeros], ignore_index=True)
    final_output_file = rf"C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part{part_num}.csv"
    final_chunk.to_csv(final_output_file, index=False)
    print(f"Saved {final_output_file} with {len(final_chunk)} samples (last part).")

print("Finished processing and saving all parts.")

print("Finished processing and saving parts.")


Total rows in the original file: 433321
Saved C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part1.csv with 80000 samples.
Saved C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part2.csv with 80000 samples.
Saved C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part3.csv with 80000 samples.
Saved C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part4.csv with 80000 samples.
Saved C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part5.csv with 80000 samples.
Saved C:\Notebooks\rrl_source\dataset_raw\train_segment_Wav2Vec2_part6.csv with 33321 samples (last part).
Finished processing and saving all parts.
Finished processing and saving parts.
